In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import operator
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
datafile = '../data/IBEX35(201301-201512).xlsx'
xl = pd.ExcelFile(datafile)
xl.sheet_names

[u'Sheet1']

In [3]:
df = xl.parse(u'Sheet1')
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2013-01-02,8447.6,8337.9,8447.6,8286.7,280.53M,0.0343
1,2013-01-03,8403.4,8375.0,8422.6,8334.3,182.28M,-0.0052
2,2013-01-04,8435.8,8411.7,8447.0,8386.7,230.12M,0.0039
3,2013-01-07,8419.0,8434.3,8485.6,8406.3,289.12M,-0.0020
4,2013-01-08,8453.0,8388.2,8493.5,8374.7,335.52M,0.0040


In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df.describe()

,Price,Open,High,Low,Change %
count,792.000000,792.000000,792.000000,792.000000,792.000000
mean,9963.164141,9965.216667,10037.700379,9878.218687,0.000415
std,1040.412907,1042.242568,1042.808739,1037.357084,0.012150
min,7553.200000,7636.600000,7674.600000,7508.400000,-0.050100
25%,9348.350000,9334.300000,9416.875000,9264.625000,-0.006800
50%,10223.250000,10227.000000,10310.500000,10129.350000,0.001000
75%,10691.600000,10700.725000,10761.575000,10611.200000,0.007600
max,11866.400000,11798.500000,11884.600000,11760.800000,0.038700


In [5]:
split_date = '2014-12-31'
train_data = df[df['Date'] <= split_date]
test_data = df[df['Date'] > split_date]
print train_data.shape, test_data.shape

(557, 7) (235, 7)


In [6]:
def universe_partition(data, d1=10, d2=10):
    x_max, x_min = data.max(axis=0), data.min(axis=0)
    std_val = data.std(axis=0)
    len_val = std_val / 10
    u_max, u_min = x_max+d2, x_min-d1 # bound of universe discourse
    u_b = np.arange(u_min, u_max, step=len_val) # cutting points
    u_s = u_b[:-1] # u1
    u_e = u_b[1:] # u2
    u_discourse = zip(u_b, u_e) # interval
    return u_discourse

In [7]:
u_discourse = universe_partition(train_data['Price'], d1=953, d2=812)
print(u_discourse)

[(6600.1999999999998, 6701.0920529569803), (6701.0920529569803, 6801.9841059139608), (6801.9841059139608, 6902.8761588709413), (6902.8761588709413, 7003.7682118279217), (7003.7682118279217, 7104.6602647849022), (7104.6602647849022, 7205.5523177418827), (7205.5523177418827, 7306.4443706988632), (7306.4443706988632, 7407.3364236558436), (7407.3364236558436, 7508.2284766128241), (7508.2284766128241, 7609.1205295698046), (7609.1205295698046, 7710.0125825267851), (7710.0125825267851, 7810.9046354837656), (7810.9046354837656, 7911.796688440746), (7911.796688440746, 8012.6887413977265), (8012.6887413977265, 8113.580794354707), (8113.580794354707, 8214.4728473116884), (8214.4728473116884, 8315.3649002686689), (8315.3649002686689, 8416.2569532256493), (8416.2569532256493, 8517.1490061826298), (8517.1490061826298, 8618.0410591396103), (8618.0410591396103, 8718.9331120965908), (8718.9331120965908, 8819.8251650535713), (8819.8251650535713, 8920.7172180105517), (8920.7172180105517, 9021.60927096753

In [8]:
def set_fuzzy_numbers(u_discourse):
    fuzzy_numbers = list()
    for i, u_i in enumerate(u_discourse):
        if (i!=0) and (i!=len(u_discourse)-1):
            u_l, u_r = u_discourse[i-1], u_discourse[i+1]
            A_l, A_r = np.mean(u_l), np.mean(u_r)
            fuzzy_numbers.append((A_l, u_i[0], u_i[1], A_r))
    return fuzzy_numbers

In [9]:
A_list = set_fuzzy_numbers(u_discourse)
print(A_list)

[(6650.6460264784901, 6701.0920529569803, 6801.9841059139608, 6852.430132392451), (6751.5380794354705, 6801.9841059139608, 6902.8761588709413, 6953.3221853494315), (6852.430132392451, 6902.8761588709413, 7003.7682118279217, 7054.214238306412), (6953.3221853494315, 7003.7682118279217, 7104.6602647849022, 7155.1062912633924), (7054.214238306412, 7104.6602647849022, 7205.5523177418827, 7255.9983442203729), (7155.1062912633924, 7205.5523177418827, 7306.4443706988632, 7356.8903971773534), (7255.9983442203729, 7306.4443706988632, 7407.3364236558436, 7457.7824501343339), (7356.8903971773534, 7407.3364236558436, 7508.2284766128241, 7558.6745030913144), (7457.7824501343339, 7508.2284766128241, 7609.1205295698046, 7659.5665560482948), (7558.6745030913144, 7609.1205295698046, 7710.0125825267851, 7760.4586090052753), (7659.5665560482948, 7710.0125825267851, 7810.9046354837656, 7861.3506619622558), (7760.4586090052753, 7810.9046354837656, 7911.796688440746, 7962.2427149192363), (7861.3506619622558,

In [10]:
def membership_evaluation(value, fuzzy_number):
    A_l, u_1, u_2, A_r = fuzzy_number
    mu = 0 # membership indication
    try:
        if np.logical_and(value>=A_l, value <u_1):
            mu = (value - A_l) / (u_1 - A_l)
        elif np.logical_and(value>=u_1, value<=u_2):
            mu = 1
        elif np.logical_and(value>u_2, value<=A_r):
            mu = (value - u_2) / (A_r - u_2)
    except ZeroDivisionError:
        mu = 0
    return mu

In [11]:
def membership_assignement(value_time_series, fuzzy_numbers):
    n_fuzzy_numbers = len(fuzzy_numbers)
    membership_list = list()
    for i, value in enumerate(value_time_series):
        value_rep = [value] * n_fuzzy_numbers
        memberships = map(lambda val, A: membership_evaluation(val, A), value_rep, fuzzy_numbers)
        max_index, _ = max(enumerate(memberships), key=operator.itemgetter(1))
        membership_list.append(max_index)
    return membership_list

In [12]:
train_data_membership_series = membership_assignement(train_data['Price'], fuzzy_numbers=A_list)
print len(train_data_membership_series)

557


In [13]:
def FLR(membership_time_series): # transition between consecutive observations
    transitions = list()
    for j, Aj in enumerate(membership_time_series):
        if j!=0:
            Ai = membership_time_series[j-1]
            transitions.append((Ai, Aj))
    return transitions

In [14]:
transition_FLR = FLR(train_data_membership_series)
print(transition_FLR)

[(17, 16), (16, 17), (17, 17), (17, 17), (17, 18), (18, 19), (19, 19), (19, 19), (19, 18), (18, 18), (18, 19), (19, 18), (18, 19), (19, 19), (19, 18), (18, 19), (19, 20), (20, 19), (19, 19), (19, 18), (18, 16), (16, 15), (15, 12), (12, 13), (13, 13), (13, 13), (13, 14), (14, 13), (13, 15), (15, 15), (15, 15), (15, 14), (14, 13), (13, 15), (15, 14), (14, 13), (13, 14), (14, 15), (15, 12), (12, 14), (14, 15), (15, 14), (14, 15), (15, 17), (17, 16), (16, 16), (16, 19), (19, 18), (18, 18), (18, 17), (17, 19), (19, 19), (19, 17), (17, 16), (16, 17), (17, 16), (16, 16), (16, 14), (14, 12), (12, 11), (11, 12), (12, 13), (13, 11), (11, 11), (11, 10), (10, 10), (10, 11), (11, 14), (14, 14), (14, 13), (13, 13), (13, 12), (12, 10), (10, 11), (11, 12), (12, 13), (13, 15), (15, 16), (16, 16), (16, 15), (15, 17), (17, 17), (17, 16), (16, 18), (18, 17), (17, 18), (18, 18), (18, 18), (18, 18), (18, 17), (17, 17), (17, 18), (18, 18), (18, 18), (18, 17), (17, 17), (17, 17), (17, 16), (16, 15), (15, 16),

In [15]:

def FLR_weight(transitions, time_series): # compute jump frequency by FLR
    jumps = map(lambda x: x[1]-x[0], transitions) # compute jumps by transitions 
    jump_time_series = zip(jumps, time_series) # assign timestamp for each jump beta^t_p,p+k
    jump_counts = defaultdict(list) 
    for key, value in jump_time_series:
        jump_counts[key].append(value) # count jump by its timestamps
    jump_counts = {key: np.sum(value) for key, value in jump_counts.items()} # sum up total time for each jump
    total_count = float(np.sum(jump_counts.values()))
    for key, value in jump_counts.iteritems(): 
        jump_counts[key] = value / total_count # normalize jumps as weights
    return jump_counts

In [16]:
first_date = train_data['Date'][0]
train_data_days = train_data['Date'].apply(lambda x: x - first_date).dt.days.tolist() # convert Timedelta to numeric days
max_day = train_data_days[-1]
# train_data_days = map(lambda d: float(d)/float(max_day), train_data_days) # normalize days
print np.sum(train_data_days)

207243


In [17]:
jump_weights = FLR_weight(transition_FLR, train_data_days[:len(transition_FLR)])
print (jump_weights)

{0: 0.42179018473234392, 1: 0.21346633416458852, 2: 0.073781565503716437, 3: 0.019921071108636175, -2: 0.068740769435634216, -4: 0.0049536353291528461, -3: 0.02072004454882212, -1: 0.17662639517710577}


In [18]:
def FRG_weight(transitions, time_series): 
    transition_time_series = zip(transitions, time_series)
    transition_groups = map(lambda x: (x[0][0], (x[0][1], x[1])), transition_time_series) 
    transition_weights = defaultdict(list)
    for key, value in transition_groups:
        transition_weights[key].append(value) # group transitions by initial state A_i
    transition_weights = {key: dict(value) for key, value in transition_weights.items()}
    for key, value in transition_weights.iteritems():
        total_weight = float(np.sum(value.values()))
        value = {k: (v/total_weight) for k, v in value.items()} # normalize weight inside each group
        transition_weights[key] = value
    return transition_weights

In [19]:
transition_weights = FRG_weight(transition_FLR, train_data_days[:len(transition_FLR)])
print transition_weights

{8: {8: 0.49855907780979825, 11: 0.5014409221902018}, 9: {8: 1.0}, 10: {10: 0.19787234042553192, 11: 0.4148936170212766, 12: 0.3872340425531915}, 11: {9: 0.16391852570320078, 10: 0.18816682832201745, 11: 0.18525703200775945, 12: 0.1901066925315228, 13: 0.1784675072744908, 14: 0.09408341416100872}, 12: {10: 0.14035087719298245, 11: 0.24696356275303644, 12: 0.26720647773279355, 13: 0.27125506072874495, 14: 0.07422402159244265}, 13: {11: 0.21664766248574688, 12: 0.21436716077537057, 13: 0.21322690992018245, 14: 0.23033067274800456, 15: 0.12542759407069556}, 14: {12: 0.13268608414239483, 13: 0.2702265372168285, 14: 0.2686084142394822, 15: 0.3284789644012945}, 15: {12: 0.06206896551724138, 13: 0.18275862068965518, 14: 0.06551724137931035, 15: 0.1793103448275862, 16: 0.23448275862068965, 17: 0.27586206896551724}, 16: {14: 0.08866442199775533, 15: 0.1717171717171717, 16: 0.265993265993266, 17: 0.2671156004489338, 18: 0.13468013468013468, 19: 0.0718294051627385}, 17: {16: 0.13369467028003612, 

In [20]:
# forecasting by fuzzy numbers
def fuzzy_add(A, B): # Proposition #1 (1)
    return tuple([np.sum(x) for x in zip(A, B)])

def fuzzy_scale(c, A): # Proposition #1 (2)
    cA = [c*a for a in A]
    if c>=0:
        return tuple(cA)
    else:
        cA.reverse()
        return tuple(cA)

def forecast_jump(i, s, A_list):
    jumps = s.keys() # possible jumps
    m = len(A_list) # number of fuzzy numbers in model
#     ips_valid = list()
#     Aip = list()
#     s_k = list()
    sA_list = list()
    sA = tuple([0]*len(A_list[0]))
    for k in jumps:
        ip = k+i
        if (ip>=0 and ip<m): # check if index is within range
#             ips_valid.append(ip) 
#             Aip.append(A[ip])
#             s_k.append(s[k])
            sA_list.append(fuzzy_scale(s[k], A_list[ip]))
    if len(sA_list)>0:
        for sa in sA_list:
            sA = fuzzy_add(sA, sa)
    return sA
    
def forecast_transition(i, w, A_list):
    wA = tuple([0]*len(A_list[0])) # default FLG relation
    if i in w.keys():
        for kj, v in w[i].iteritems():
            wA = fuzzy_add(wA, fuzzy_scale(v, A_list[kj]))
    return wA

def forecast_price(As, Aw, gamma=0.1):
    if gamma<0 or gamma>1:
        raise ValueError("gamma should be between 0.0 and 1.0 (inclusive on both ends)")
    Ai_pred = fuzzy_add(fuzzy_scale(gamma, As), fuzzy_scale(1-gamma, Aw))
    return np.mean(Ai_pred)

In [21]:
def get_membership(value, fuzzy_numbers):
    n_fuzzy_numbers = len(fuzzy_numbers)
    membership_index = 0
    value_rep = [value] * n_fuzzy_numbers
    memberships = map(lambda val, Ai: membership_evaluation(val, Ai), value_rep, fuzzy_numbers)
    membership_index, _ = max(enumerate(memberships), key=operator.itemgetter(1))
    return membership_index

In [23]:
# fit training data
from sklearn.metrics import mean_squared_error
train_features = train_data['Price'].tolist()[:-1]
train_prices = train_data['Price'].tolist()[1:]
fit_prices = list()
for price in train_features:
    index = get_membership(price, A_list)
    next_price = forecast_price(forecast_jump(index, jump_weights, A_list), forecast_transition(index, transition_weights, A_list), gamma=0.9)
    fit_prices.append(next_price)
print np.sqrt(mean_squared_error(train_prices, fit_prices))

110.127292194


In [24]:
# forecasting test data
test_features = [train_data['Price'].tolist()[-1]] + test_data['Price'].tolist()[:-1]
actual_prices = test_data['Price'].tolist()
pred_prices = list()
for price in test_features:
    index = get_membership(price, A_list)
    next_price = forecast_price(forecast_jump(index, jump_weights, A_list), forecast_transition(index, transition_weights, A_list), gamma=0.9)
    pred_prices.append(next_price)
print np.sqrt(mean_squared_error(actual_prices, pred_prices))

822.977236429


In [25]:
# from pprint import pprint
# pprint(zip(actual_prices, pred_prices))

In [27]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
import plotly.plotly as py
py.sign_in('imaginationsuper', 'PQj7gXzsTJFqNKsXkWMT')
init_notebook_mode(connected=True)
df = pd.DataFrame(train_data[1:]['Date'])
df['data training'] = pd.Series(train_prices).values
df['data fitting'] = pd.Series(fit_prices).values
trace_fit = go.Scatter(
                x=df['Date'],
                y=df['data fitting'],
                name = "data fitting",
                line = dict(color = 'blue'),
                opacity = 0.8)
trace_train = go.Scatter(
                x=df['Date'],
                y=df['data training'],
                name = "data training",
                line = dict(color = 'black'),
                opacity = 0.8)
data = [trace_fit, trace_train]
layout = dict(
    title = "IBEX35 Price Fitting",
    xaxis = dict(
        range = ['2013-01-02','2014-12-31'])
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "IBEX35_fitting")

In [28]:
df = pd.DataFrame(test_data['Date'])
df['data testing'] = pd.Series(actual_prices).values
df['forecasting'] = pd.Series(pred_prices).values
trace_forecast = go.Scatter(
                x=df['Date'],
                y=df['forecasting'],
                name = "forecasting",
                line = dict(color = 'green'),
                opacity = 0.8)
trace_actual = go.Scatter(
                x=df['Date'],
                y=df['data testing'],
                name = "data testing",
                line = dict(color = 'black'),
                opacity = 0.8)
data = [trace_forecast, trace_actual]
layout = dict(
    title = "IBEX35 Price Forecasting",
    xaxis = dict(
        range = ['2015-01-02','2015-12-01'])
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "IBEX35_forecasting")